In [1]:
import numpy as np

In [2]:
array = []
with open('min15.txt') as file:
    for line in file.readlines():
        string = line.split(',') 
        del string[:2]
        del string[-1]
        array.append(string)
#         print(line)

In [3]:
data = np.asarray(array[1:])
heads = array[0]

In [4]:
heads

['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>']

In [5]:
time = data[:,1]

In [6]:
time

array(['234500', '000000', '001500', ..., '220000', '221500', '223000'],
      dtype='<U10')

In [7]:
minutes = np.asarray([element[2:4] for element in time]).reshape(-1,1).astype(np.int32)
hours = np.asarray([element[:2] for element in time]).reshape(-1,1).astype(np.int32)

In [8]:
date = data[:,0]

In [9]:
year = np.asarray([element[:4] for element in date]).reshape(-1,1).astype(np.int32)
month = np.asarray([element[4:6] for element in date]).reshape(-1,1).astype(np.int32)
day = np.asarray([element[6:] for element in date]).reshape(-1,1).astype(np.int32)

In [10]:
data = np.delete(data,1,1)
data = np.delete(data,0,1)
# data = data.astype(np.float32)
data = np.hstack((year,month,day,hours,minutes,data))

In [11]:
train_data = data[:,5]

In [12]:
ds_x = []
ds_y = []
days_hist= 10
days_predict = 10
period = 24 * 4 * days_hist
period_predict = 24 * 4 * days_predict
for num,line in enumerate(train_data[:-(period+period_predict)]):
    ds_x.append(train_data[num:num+period_predict].reshape((train_data[num:num+period_predict].size)))
for line in train_data[period+period_predict:]:
    ds_y.append(line)

In [ ]:
ds_x = np.asarray(ds_x, dtype = np.float16)
ds_y = np.asarray(ds_y, dtype = np.float16).reshape(-1,1)

In [ ]:
idx = np.array(range(len(ds_y)))

In [ ]:
np.random.shuffle(idx)

In [ ]:
ds_x = ds_x[idx]
ds_y = ds_y[idx]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
ds_x_train, ds_x_test, ds_y_train, ds_y_test =  train_test_split(ds_x, ds_y, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
linreg = LinearRegression(n_jobs = -1)
dtreg = DecisionTreeRegressor()
knreg = KNeighborsRegressor(n_jobs = -1)
rfr = RandomForestRegressor(n_jobs = -1)
mlpreg = MLPRegressor()

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
estimators = [linreg, dtreg, knreg, rfr, mlpreg]
names = []
scores = []
for estimator in estimators:
    names.append(str(estimator).split(sep = '(')[0])
    estimator.fit(ds_x_train,ds_y_train)
    scores.append(mean_squared_error(ds_y_train, estimator.predict(ds_x_train), squared = False))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [52]:
linreg.fit(ds_x_train,ds_y_train.reshape(-1))
# mean_squared_error(linreg.reshape(-1), rfr.predict(ds_x_train), squared = False)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [44]:
from sklearn.model_selection import cross_val_score

In [58]:
print(cross_val_score(linreg, ds_x_train, ds_y_train, cv=10, scoring='r2', n_jobs = -1))
print(cross_val_score(linreg, ds_x_train, ds_y_train, cv=10, scoring='neg_mean_squared_error', n_jobs = -1))
print(cross_val_score(linreg, ds_x_train, ds_y_train, cv=10, scoring='neg_mean_absolute_error', n_jobs = -1))

[0.88664409 0.88669684 0.88637292 0.88859074 0.88393866 0.88465375
 0.88799066 0.8855487  0.88294902 0.88594872]
[-2.59708834 -2.57803416 -2.67654371 -2.54779482 -2.63913155 -2.60507774
 -2.5442276  -2.58674788 -2.64309764 -2.54552197]
[-1.16869128 -1.17226279 -1.19477534 -1.16783404 -1.18711817 -1.16681683
 -1.16412389 -1.17272604 -1.18133962 -1.16814637]


In [59]:
print(cross_val_score(knreg, ds_x_train[:40000], ds_y_train[:40000], cv=5, scoring='r2', n_jobs = -1))
print(cross_val_score(knreg, ds_x_train[:40000], ds_y_train[:40000], cv=5, scoring='neg_mean_squared_error', n_jobs = -1))
print(cross_val_score(knreg, ds_x_train[:40000], ds_y_train[:40000], cv=5, scoring='neg_mean_absolute_error', n_jobs = -1))

[0.99945817 0.99945181 0.99952326 0.99948132 0.99945295]
[-0.01237488 -0.01257324 -0.01118469 -0.01179504 -0.01255035]
[-0.06286621 -0.06176758 -0.06005859 -0.06030273 -0.06246948]


In [60]:
knreg.fit(ds_x_train,ds_y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                    weights='uniform')

In [61]:
predict = knreg.predict(ds_x_test)

In [65]:
print(cross_val_score(mlpreg, ds_x_train[:40000], ds_y_train[:40000], cv=5, scoring='r2', n_jobs = -1))
print(cross_val_score(mlpreg, ds_x_train[:40000], ds_y_train[:40000], cv=5, scoring='neg_mean_squared_error', n_jobs = -1))
print(cross_val_score(mlpreg, ds_x_train[:40000], ds_y_train[:40000], cv=5, scoring='neg_mean_absolute_error', n_jobs = -1))

[-1.21602165 -1.23477057  0.7172149   0.87536919  0.3488834 ]
[ -3.23989331  -5.54775123  -5.96836266 -14.95316666 -26.14032788]
[-3.15315731 -1.2310353  -1.23331744 -3.98694332 -3.68628095]


In [67]:
rfr.max_depth = 10

In [ ]:
print(cross_val_score(rfr, ds_x_train[:40000], ds_y_train[:40000], cv=5, scoring='r2', n_jobs = -1))
print(cross_val_score(rfr, ds_x_train[:40000], ds_y_train[:40000], cv=5, scoring='neg_mean_squared_error', n_jobs = -1))
print(cross_val_score(rfr, ds_x_train[:40000], ds_y_train[:40000], cv=5, scoring='neg_mean_absolute_error', n_jobs = -1))